In [2]:
# Dependencies
import numpy as np
import pandas as pd
import datetime

# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [3]:
engine = create_engine("sqlite:///hawaii.sqlite")

# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [4]:
# Create our session (link) from Python to the DB
session = Session(engine)

## D1: Determine the Summary Statistics for June

In [5]:
# 1. Import the sqlalchemy extract function.
from sqlalchemy import extract

# 2. Write a query that filters the Measurement table to retrieve the temperatures for the month of June. 
results = session.query(Measurement.tobs).\
    where(extract('month', Measurement.date) == 6).all()

In [6]:
#  3. Convert the June temperatures to a list.
june_temps = list(np.ravel(results))

In [7]:
# 4. Create a DataFrame from the list of temperatures for the month of June. 
june_temps_df = pd.DataFrame(june_temps, columns = ["Temp"])


In [8]:
# 5. Calculate and print out the summary statistics for the June temperature DataFrame.
june_temps_df.describe()

,Temp
count,1700.000000
mean,74.944118
std,3.257417
min,64.000000
25%,73.000000
50%,75.000000
75%,77.000000
max,85.000000


## D2: Determine the Summary Statistics for December

In [9]:
# 6. Write a query that filters the Measurement table to retrieve the temperatures for the month of December.
results = session.query(Measurement.tobs).\
    where(extract('month', Measurement.date) == 12).all()

In [10]:
# 7. Convert the December temperatures to a list.
dec_temps = list(np.ravel(results))

In [11]:
# 8. Create a DataFrame from the list of temperatures for the month of December. 
dec_temps_df = pd.DataFrame(dec_temps, columns = ["Temp"])

In [12]:
# 9. Calculate and print out the summary statistics for the Decemeber temperature DataFrame.
dec_temps_df.describe()

,Temp
count,1517.000000
mean,71.041529
std,3.745920
min,56.000000
25%,69.000000
50%,71.000000
75%,74.000000
max,83.000000


In [20]:
# specify columns and run query
cols = [extract('month', Measurement.date), func.min(Measurement.tobs), func.round(func.avg(Measurement.tobs), 1), func.max(Measurement.tobs)]

results = session.query(*cols)\
    .group_by(extract('month', Measurement.date)).all()

# convert to dataframe, change month numbers to names, display
temps_by_month = pd.DataFrame(results, columns=['Month', 'Min Temp', 'Avg Temp', 'Max Temp'])

temps_by_month['Month'] = [datetime.datetime.strptime(str(month), "%m").strftime("%B") for month in temps_by_month['Month']]

temps_by_month.set_index('Month', inplace=True)

temps_by_month


,Min Temp,Avg Temp,Max Temp
Month,,,
January,54.0,68.7,81.0
February,56.0,69.4,83.0
March,56.0,70.1,82.0
April,53.0,72.4,84.0
May,63.0,73.7,87.0
June,64.0,74.9,85.0
July,61.0,76.1,87.0
August,65.0,76.4,87.0
September,64.0,76.2,87.0
